In [30]:
import json 
import requests
import time
import urllib.request
import math

TOKEN = "865979318:AAGfil3jAjgM8Qzl9m_j6uvksmPC7U7RkM8"
URL = "https://api.telegram.org/bot{}/".format(TOKEN)
URL_FILE = "https://api.telegram.org/file/bot{}/".format(TOKEN)


def get_url(url):
    response = requests.get(url)
    content = response.content.decode("utf8")
    return content


def get_json_from_url(url):
    content = get_url(url)
    js = json.loads(content)
    return js


def get_updates():
    url = URL + "getUpdates"
    js = get_json_from_url(url)
    return js


def get_last_chat_id_and_text(updates):
    num_updates = len(updates["result"])
    last_update = num_updates - 1
    global tipekonten
    konten = [None,None]
    try:
        konten = updates["result"][last_update]["message"]["photo"][2]["file_id"]
        tipekonten="gambar"
    except KeyError:
        try:
            konten = updates["result"][last_update]["message"]["text"]
            tipekonten="text"
        except KeyError:
            try:
                konten[0] = updates["result"][last_update]["message"]["location"]["latitude"]
                konten[1] = updates["result"][last_update]["message"]["location"]["longitude"]
                tipekonten="location"
            except KeyError:
                True
    
    chat_id = updates["result"][last_update]["message"]["chat"]["id"]
    
    return ( tipekonten,konten, chat_id)


def send_message(text, chat_id):
    url = URL + "sendMessage?text={}&chat_id={}".format(text, chat_id)
    get_url(url)
    
def send_file(file_id, chat_id):
    url = URL + "sendPhoto?photo={}&chat_id={}".format(file_id, chat_id)
    get_url(url)
    
def save_file(file_id, chat_id):
    url = URL + "getFile?file_id={}".format(file_id)
    js = get_json_from_url(url)
    file_path = js["result"]["file_path"]
    url = URL_FILE + file_path
    testfile = urllib.request.urlretrieve(url,str(file_id)+".jpg")
    send_message("image downloaded",chat_id)
    
'''
def prosedur_input():
    input_tiket=False
    while True:
        tipekonten, konten, chat = get_last_chat_id_and_text(get_updates())
        if((input_tiket=False)&&(last_textchat!=(konten,chat))):
            send_message("Silahkan Input Nomor Tiket", chat)
            if(tipekonten == 'text'):
                input_tiket=True
        if((input_tiket==True)&&(last_textchat!=(konten,chat)):
            send_message("Silahkan Input Gambar Beserta Caption", chat)
        last_textchat = (konten, chat)
        while(tipekonten == 'text'):
            last_textchat = (konten, chat)
        if(tipekonten="text"):
            
'''    
def calc_distance(lat1, lon1):
    lat2=-6.229697
    lon2=106.816049
    R = 6371e3  #metres
    o1 = math.radians(lat1)
    o2 = math.radians(lat2)
    deltaφ = math.radians(lat2-lat1)
    deltaλ = math.radians(lon2-lon1)

    a = math.sin(deltaφ/2) * math.sin(deltaφ/2) + math.cos(o1) * math.cos(o2) * math.sin(deltaλ/2) * math.sin(deltaλ/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c
    return(d)

def main():
    last_textchat = (None, None)
    while True:
        tipekonten, konten, chat = get_last_chat_id_and_text(get_updates())
        if (konten, chat) != last_textchat:
            if(tipekonten=="text"):
                send_message(konten, chat)
            elif(tipekonten=="gambar"):
                save_file(konten, chat)
            elif(tipekonten=="location"):
                msg = str(round(calc_distance(konten[0],konten[1]))) + " m"
                send_message(msg,chat)
            last_textchat = (konten, chat)
        time.sleep(0.5)


if __name__ == '__main__':
    main()


KeyboardInterrupt: 